In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"

In [ ]:
import time
import numpy as np

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext(master = "local", appName = "App").getOrCreate()
spark = SparkSession.builder.getOrCreate()

print(sc, sc.version, spark)

# Load data

In [ ]:
trainPath = '/content/drive/MyDrive/Colab\ Notebooks/Projects/kaggle/NLP_with_Disaster_Tweets/nlp-getting-started/train.csv'
trainData = spark.read.format('csv').options(header='true', inferSchema='true', multiLine=True).load(trainPath)

In [ ]:
testPath = '/content/drive/MyDrive/Colab\ Notebooks/Projects/kaggle/NLP_with_Disaster_Tweets/nlp-getting-started/test.csv'
testData = spark.read.format('csv').options(header='true', inferSchema='true', multiLine=True).load(testPath)

# Pipeline

In [ ]:
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable  
from pyspark.ml import Pipeline 
import pyspark.sql.functions as F
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, Word2Vec, StringIndexer,OneHotEncoder, VectorAssembler, RobustScaler

In [ ]:
class FillNanTransformer(Transformer, HasInputCols, DefaultParamsReadable, DefaultParamsWritable):
    nanReplacement = Param(Params._dummy(), "nanReplacement", "nanReplacement", typeConverter=TypeConverters.toString)
    
    @keyword_only
    def __init__(self, inputCols=None, nanReplacement=None):
        super(FillNanTransformer, self).__init__()
        self._setDefault(nanReplacement="")
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols=None, nanReplacement=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def getNanReplacement(self):
        return self.getOrDefault(self.nanReplacement)
    
    def _transform(self, dataset):
        nanReplacement = self.getNanReplacement()
        dataset = dataset.na.fill(value=nanReplacement,subset=self.getInputCols())
        return dataset
    
class RemovePatternTransformer(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    pattern = Param(Params._dummy(), "pattern", "pattern", typeConverter=TypeConverters.toString)
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, pattern=None):
        super(RemovePatternTransformer, self).__init__()
        self._setDefault(pattern="")
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, pattern=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def getPattern(self):
        return self.getOrDefault(self.pattern)
    
    def _transform(self, dataset):
        pattern = self.getPattern()
        dataset = dataset.withColumn(self.getOutputCol(), F.regexp_replace(F.col(self.getInputCol()), pattern, ""))
        return dataset
    
class CheckPatternTransformer(Transformer, HasInputCol, HasOutputCol):
    pattern = Param(Params._dummy(), "pattern", "pattern", typeConverter=TypeConverters.toString)
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, pattern=None):
        super(CheckPatternTransformer, self).__init__()
        self._setDefault(pattern="")
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, pattern=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def getPattern(self):
        return self.getOrDefault(self.pattern)
    
    def _transform(self, dataset):
        pattern = self.getPattern()
        dataset = dataset.withColumn(self.getOutputCol(), F.when(F.col(self.getInputCol()).rlike(pattern),1.).otherwise(0.))
        return dataset
    
class GetLengthTransformer(Transformer, HasInputCols, HasOutputCols):
    @keyword_only
    def __init__(self, inputCols=None, outputCols=None):
        super(GetLengthTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols=None, outputCols=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    
    def _transform(self, dataset):
        for inputCol, outputCol in zip(self.getInputCols(), self.getOutputCols()):
            dataset = dataset.withColumn(outputCol, F.length(inputCol))
        return dataset
    
class ConcatenateTransformer(Transformer, HasInputCols, HasOutputCol):
    @keyword_only
    def __init__(self, inputCols=None, outputCol=None):
        super(ConcatenateTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, dataset):
        dataset = dataset.withColumn(self.getOutputCol(), F.col(self.getInputCols()[0]))
        for colName in self.getInputCols()[1:]:
            dataset = dataset.withColumn(self.getOutputCol(), 
                F.concat_ws('@', F.col(self.getOutputCol()), F.col(colName)))
        return dataset

In [ ]:
fillNanTransformer = FillNanTransformer(inputCols=["keyword", "location"], nanReplacement="$")
textFillNanTransformer = FillNanTransformer(inputCols=["text"], nanReplacement="")

#---
removeUrlTransformer = RemovePatternTransformer(inputCol="text", outputCol="textNoUrl", pattern="(https?://\S+)")
regexTokenizer = RegexTokenizer(inputCol="textNoUrl", outputCol="textArrayWord", pattern="\\W")

stopWordsRemover = StopWordsRemover(inputCol="textArrayWord", outputCol="textNoSW")
word2Vec = Word2Vec(vectorSize=50, windowSize=10, minCount=0, inputCol="textNoSW", outputCol="textVec")

#---
keywordIndexer = StringIndexer(inputCol="keyword", outputCol="keywordIndex", handleInvalid="keep")
locationIndexer = StringIndexer(inputCol="location", outputCol="locationIndex", handleInvalid="keep")
checkUrlTransformer = CheckPatternTransformer(inputCol="text", outputCol="textIsContainedUrl", pattern="(https?://\S+)")

getLengthTransformer = GetLengthTransformer(inputCols=["keyword","textNoUrl"], outputCols=["keywordLen", "textNoUrlLen"])

oneHotEncoder = OneHotEncoder(inputCols=["keywordIndex", "locationIndex", "textIsContainedUrl"],
                              outputCols=["keywordVec", "locationVec", "textIsContainedUrlVec"],
                              handleInvalid="keep")

#---
concatStringTransformer = ConcatenateTransformer(inputCols=["keyword", "location", "textNoUrl"], outputCol="concatString")
concatStringRegexTokenizer = RegexTokenizer(inputCol="concatString", outputCol="concatStringArrayWord", pattern="\\W")
concatStringStopWordsRemover = StopWordsRemover(inputCol="concatStringArrayWord", outputCol="concatStringArrayWordNoSW")

concatStringWord2Vec = Word2Vec(vectorSize=50, windowSize=10, minCount=0, inputCol="concatStringArrayWord", outputCol="concatStringVec")
concatStringNoSWWord2Vec = Word2Vec(vectorSize=50, windowSize=10, minCount=0, inputCol="concatStringArrayWordNoSW", outputCol="concatStringNoSWVec")

#---

discreteFeaturesAssembler = VectorAssembler(inputCols=["keywordVec", "locationVec", "textIsContainedUrlVec",
                                                      "keywordLen", "textNoUrlLen"], 
                                            outputCol="discreteFeatures")

discreteAndTextFeaturesAssembler = VectorAssembler(inputCols=["discreteFeatures", "textVec"],
                                                   outputCol="discreteAndTextFeatures")

discreteFeaturesRobustScaler = RobustScaler(inputCol="discreteFeatures", outputCol="discreteFeaturesScale",
                                            withScaling=True, withCentering=True, lower=0.25, upper=0.75)

discreteAndTextFeaturesRobustScaler = RobustScaler(inputCol="discreteAndTextFeatures", outputCol="discreteAndTextFeaturesScale",
                                                   withScaling=True, withCentering=True, lower=0.25, upper=0.75)

In [ ]:
preprocessingPipeline = Pipeline(stages=[fillNanTransformer, textFillNanTransformer,
                                         concatStringTransformer, concatStringRegexTokenizer, concatStringStopWordsRemover, 
                                         concatStringNoSWWord2Vec
                                        ])

# Train

In [ ]:
preprocessingModel = preprocessingPipeline.fit(trainData)

trainDataPreprocessed = preprocessingModel.transform(trainData)
testDataPreprocessed = preprocessingModel.transform(testData)

In [ ]:
trainSet, validSet = trainDataPreprocessed.randomSplit([0.9, 0.1], seed=12345)

In [ ]:
featuresCol = "concatStringNoSWVec"
labelCol = "target"

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier, LinearSVC

evaluator = BinaryClassificationEvaluator(labelCol=labelCol, rawPredictionCol="prediction", metricName="areaUnderROC")

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator

algorithm = GBTClassifier(featuresCol=featuresCol, labelCol=labelCol, maxIter=100)
paramGrid = ParamGridBuilder()
.addGrid(algorithm.minInstancesPerNode, np.arange(1,10,1))\
.addGrid(algorithm.maxDepth, np.arange(5,50,5))\
.addGrid(algorithm.maxBins, np.arange(30,100,10))\
.build()


validator = CrossValidator(estimator=algorithm,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           numFolds=3)

In [ ]:
validatorModel = validator.fit(trainData)

print(validatorModel.avgMetrics)

model = validatorModel.bestModel

In [ ]:
predictionTest = model.fit(trainDataPreprocessed).transform(testDataPreprocessed)
predLabelTest = np.array(predictionTest.select('prediction').collect()).squeeze()

In [ ]:
submission = pd.read_csv('nlp-getting-started/sample_submission.csv')
submission['target'] = submission['target'] + predLabelTest.astype(int)
submission.to_csv('submission.csv', index=False)

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "LightGbm-Concat"